In [1]:
import sys
sys.path.append("../..")

import os
import pickle
import random


import pandas as pd
from pandas import DataFrame

from app.components.extract_data.extract_data import getDfPerson, getDfConsumo, getDictV9001toGroupPt, getDictV9001toGroupEn
from app.components.simple_types import Nutrition, State
from app.components.search.papa_search import SearchResult, papaSingleSeach, fitness, SearchResult

Loading basic data
Loading basic dictionary
Finish!


In [2]:
datasetPath = os.getcwd() + "/../../datasets/"
datasetPicklePath = os.getcwd() + "/../../datasets/pickle/"

In [3]:
dfConsumo = getDfConsumo()
dfPerson = getDfPerson()

In [4]:
len(dfPerson["ESTRATO_POF"].unique())

575

In [5]:
class PersonInformation:
    def __init__(self, person:str):
        # Id da pessoa no POF
        self.id:str = person
        # Dieta inicial da pessoa
        self.initialDiet:State = State.getStateByPersonId(person)        
        # Dieta inicial do passo atual
        self.actualDiet:State = State.getStateByPersonId(person) 
        # Nutriçao recomendada para a pessoa
        self.targetNutrition= Nutrition.idealNutritionByPersonId(person) 

In [6]:
class DietAgent:
    def __init__(self, person:str):
        self.personInformation:PersonInformation = PersonInformation(person)
        self.optimizedDiets:list[State] = [State.getStateByPersonId(person)]

In [7]:
class DietAllyEnemy:
    def __init__(self, person:str):
        self.enemy:int = -1
        self.ally:int = -1
        self.bestDiet:State = State.getStateByPersonId(person) 
        self.bestDietFitness:float = None
    

In [8]:
class HealthAgent:
    def __init__(self):
        self.personDietAllyEnemmy:dict[str, DietAllyEnemy] = {}
        self.dietAgents:dict[str, DietAgent] = {}
        
    def add_person(self, person:str): 
        self.personDietAllyEnemmy[person] = DietAllyEnemy(person)
        self.dietAgents[person] = DietAgent(person)
        
    def assign_social(self):
        allies = list(self.dietAgents.keys())
        enemies = list(self.dietAgents.keys())
        random.shuffle( allies )
        random.shuffle( enemies )
        
        people = list(self.dietAgents.keys())
        for id, person in enumerate(people):
            self.personDietAllyEnemmy[person].ally = allies[id]
            self.personDietAllyEnemmy[person].enemy = enemies[id]
            
    def updateBest(self, person:str, candidateBest:State, nutritionFactor:float=1.2, distanceFactor:float=0.1):
        self.personDietAllyEnemmy[person]
        
        if self.personDietAllyEnemmy[person].bestDietFitness == None:
            self.personDietAllyEnemmy[person].bestDietFitness = fitness(
                state=self.personDietAllyEnemmy[person].bestDiet,
                initialState=self.dietAgents[person].personInformation.actualDiet,
                targetNutrition=self.dietAgents[person].personInformation.targetNutrition,
                nutritionFactor=nutritionFactor,
                distanceFactor=distanceFactor, 
            )
        
        candidateFitness = fitness(
            state=candidateBest,
            initialState=self.dietAgents[person].personInformation.actualDiet,
            targetNutrition=self.dietAgents[person].personInformation.targetNutrition,
            nutritionFactor=nutritionFactor,
            distanceFactor=distanceFactor, 
        )
        
        if( self.personDietAllyEnemmy[person].bestDietFitness > candidateFitness ):
            self.personDietAllyEnemmy[person].bestDietFitness = candidateFitness
            self.personDietAllyEnemmy[person].bestDiet = candidateBest
            

In [9]:
personID = "110000016#7#1#5"

In [10]:
healthAgent = HealthAgent()
dietAgent = DietAgent(personID)

In [11]:
healthAgent.add_person(personID)

In [12]:
#Find people 
dfPerson = getDfPerson()
stratos = dfPerson[ (dfPerson["AGE"]==25) & (dfPerson["GENDER"]=="male") ]["ESTRATO_POF"].to_list()
quantity = 3
male25 = dfPerson[ (dfPerson["AGE"]==25) & (dfPerson["GENDER"]=="male") ]["PESSOA"].to_list()[:quantity]
male60 = dfPerson[ (dfPerson["AGE"]==60) & (dfPerson["GENDER"]=="male") ]["PESSOA"].to_list()[:quantity]
female25 = dfPerson[ (dfPerson["AGE"]==25) & (dfPerson["GENDER"]=="female") ]["PESSOA"].to_list()[:quantity]
female60 = dfPerson[ (dfPerson["AGE"]==60) & (dfPerson["GENDER"]=="female") ]["PESSOA"].to_list()[:quantity]

In [13]:
def multiAgentSearch(
        people:list[str], 
        resultFileName:str, 
        maxInterations:int = 3,
        maxSteps:int = 10,
        maxSearchSteps:int = 10,
        nutritionFactor:float = 1.2,
        distanceFactor:float = 0.1, 
        verbose:bool=False) -> list[list[SearchResult]]:
    try:
        with open("./output/" + resultFileName, "rb") as file:
            return pickle.load(file)
    except:
        
        healthAgent = HealthAgent()
        for person in people:
            healthAgent.add_person(person)
        healthAgent.assign_social()
        
        results = []
        phases = ["corward", "hero"]
        
        for interation in range(maxInterations):
            # RUN multiAgentSearch
            for step in range(maxSteps):
                # operator:function = None
                if phases[step%2] == "corward":
                #     # TODO: Fazer o operador covarde
                #     operator = cowardOperator
                    pass
                elif phases[step%2] == "hero":
                #     # TODO: Fazer o operador heroi
                #     operator = heroOperator
                    pass
                                
                for agent in healthAgent.dietAgents.values():
                    person = agent.personInformation.id
                    
                    result = papaSingleSeach(
                                personID=person, 
                                verbose=False, 
                                unit=50, 
                                max_unit=3, 
                                max_population_set=20, 
                                max_population_selected=10, 
                                expansion_set=20, 
                                expansion_select=5, 
                                max_steps=maxSearchSteps, 
                                nutritionFitness=Nutrition.absDifferenceNegativePenalty, 
                                nutritionFactor=nutritionFactor,
                                distanceFitness=State.squareDifference,
                                distanceFactor = distanceFactor,
                                preserve_best=True,
                                crossover=0.1, 
                                initialState=agent.personInformation.actualDiet,
                                initialPopulation=agent.optimizedDiets,
                            )

                    healthAgent.updateBest(
                            person=person,
                            candidateBest=result.finalMeal,
                        )
                        
                       
            if verbose:
                print("### Interation", interation, "####\n")
                for agent in healthAgent.dietAgents.values():
                    person = agent.personInformation.id 
                    
                    print("Agent:", person)
                    print("Initial diet:", agent.personInformation.actualDiet)
                    print("Final diet:", healthAgent.personDietAllyEnemmy[person].bestDiet)
                    initialFitness = fitness(
                        agent.personInformation.actualDiet,
                        agent.personInformation.actualDiet,
                        agent.personInformation.targetNutrition,
                        nutritionFactor=nutritionFactor,
                        distanceFactor=distanceFactor,
                    )
                    print("Initial fitness:", initialFitness)
                    print("Final fitness:", healthAgent.personDietAllyEnemmy[person].bestDietFitness)
                    print("#############")
                    
                
            
            
            # # Build Results
            # for i, person in enumerate(people):
            #     results[interation].append(
            #                 SearchResult(
            #                     [person],
            #                     healthAgent.dietAgents[i].personInformation.initialDiet,
            #                     healthAgent.personDietAllyEnemmy[i].diet,
            #                 ))
        
        # with open("./output/" + resultFileName, "wb") as file:
        #     pickle.dump(results, file) 
        
        return results
            

In [14]:
print(male25)
results:list[list[SearchResult]] = []
results = multiAgentSearch(
                people=male25, 
                resultFileName="male25_100_agent.pickle", 
                maxInterations=2, 
                maxSteps=2, 
                maxSearchSteps=1,
                verbose=True
            )

print(results)

['110004584#15#1#1', '110010188#3#1#4', '110012387#6#1#3']
### Interation 0 ####

Agent: 110004584#15#1#1
Initial diet: State: {8000101: 0.0, 7901602: 0.0, 8501303: 0.0, 6906602: 0.0, 6300101: 490.0, 7112802: 0.0, 6303102: 840.0, 6601805: 0.0, 8201004: 4000.0, 8570328: 0.0, 7108902: 0.0, 8510601: 0.0, 8200101: 0.0, 8500903: 0.0, 7004301: 0.0, 7004802: 0.0, 8002201: 0.0, 8400101: 0.0, 8004801: 0.0, 8500401: 0.0, 6501516: 0.0, 6300113: 0.0, 7800302: 0.0, 8500605: 0.0, 6902910: 0.0, 6803001: 0.0, 6901701: 0.0, 8201005: 0.0, 8002254: 0.0, 6900702: 0.0, 8503801: 0.0, 8300101: 0.0, 8501302: 0.0, 7109101: 0.0, 8511401: 0.0, 8280001: 0.0, 7800401: 0.0, 6701704: 0.0, 6801101: 0.0, 6300201: 0.0, 7803301: 0.0, 6400802: 0.0, 6802601: 0.0, 8216301: 0.0, 7104102: 0.0, 8502204: 0.0, 8502202: 0.0, 8579146: 0.0, 6501415: 0.0, 8500416: 0.0, 8003801: 0.0, 6901002: 0.0, 8002225: 0.0, 7104301: 0.0, 8570329: 0.0, 6503401: 0.0, 8102205: 0.0, 8500202: 0.0, 8102101: 0.0, 8002246: 0.0, 8200301: 0.0, 6501401: 0.

In [15]:
d = {
    2:4,
    3:9,
    4:16
}

print(list(d.keys()))

for a, b in d.items():
    print(a, b)

[2, 3, 4]
2 4
3 9
4 16
